# Задачи на основы SQL

**Будем использовать связку sqlite3**

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect('db')

In [3]:
df = pd.read_csv('german_credit_augmented.csv')
df.head(5)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


Заметим, что дата залилась как текст, и в sql она тогда тоже будет представлена текстом, чтобы это избежать нужно сразу перeводить в дату.

In [4]:
df['contract_dt'] = pd.to_datetime(df['contract_dt'], format='%Y-%m-%d %H:%M:%S')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   age               1000 non-null   int64         
 1   sex               1000 non-null   object        
 2   job               1000 non-null   int64         
 3   housing           1000 non-null   object        
 4   saving_accounts   817 non-null    object        
 5   checking_account  606 non-null    object        
 6   credit_amount     1000 non-null   int64         
 7   duration          1000 non-null   int64         
 8   purpose           1000 non-null   object        
 9   default           1000 non-null   int64         
 10  contract_dt       1000 non-null   datetime64[ns]
 11  client_id         1000 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(5)
memory usage: 93.9+ KB


In [6]:
# теперь зальем нашу таблицу в базу данных

df.to_sql('german_credit', con, index=False, if_exists='replace')

1000

**Чтобы облегчить обращение к таблице напишем функцию select** 

In [7]:
def select(sql):
    return pd.read_sql(sql, con)

**Задача 1. Подсчет наблюдений по конкретному столбцу**

In [8]:
sql = '''select count(t.saving_accounts) as cnt from german_credit as t'''
select(sql)

,cnt
0,817


In [9]:
# как видим, count по конкретному столбцу не считает пропуски

**Задача 2. Применение like. Подсчитать количество вхождений строк по группам наблюдений.**

In [10]:
# создам искусственный пример для демонстрации

t = pd.DataFrame({'purpose' : ['машина', 'в машине', 'у машины', 'автомобиль', 'на новый год 2007'], 
                  'amount' : [1000, 400, 600, 700, 1500]
})

t

,purpose,amount
0,машина,1000
1,в машине,400
2,у машины,600
3,автомобиль,700
4,на новый год 2007,1500


In [11]:
t.to_sql('credit', con, index=False, if_exists='replace')
sql = '''select * from credit'''
select(sql)

,purpose,amount
0,машина,1000
1,в машине,400
2,у машины,600
3,автомобиль,700
4,на новый год 2007,1500


In [12]:
sql = '''select t.* from credit as t 
where t.purpose like '%машин%' or t.purpose = 'автомобиль' '''
select(sql)

,purpose,amount
0,машина,1000
1,в машине,400
2,у машины,600
3,автомобиль,700


**Задача 3. Теперь задача отобрать все кредиты, где цена больше 500**

In [13]:
sql = '''select t.* from credit as t
where (t.purpose like '%машин%' or t.purpose like '%автомобиль%') and t.amount > 500'''
select(sql)

,purpose,amount
0,машина,1000
1,у машины,600
2,автомобиль,700


**Задача 4. Компактное проставление флага. Значениям больше 1000 поставить 1**

In [14]:
sql = '''select t.credit_amount, 
iif(t.credit_amount > 1000, 1, 0) as greater from german_credit as t'''
select(sql)

# то есть, такой способ тоже работает как вариант



,credit_amount,greater
0,3074,1
1,1344,1
2,936,0
3,1393,1
4,776,0
...,...,...
995,2600,1
996,4455,1
997,6403,1
998,5003,1


**Задача 5. Построить сводную таблицу по полу и среднему размеру кредита**

In [15]:
sql = '''select t.sex, count(1), avg(t.credit_amount) as avg from german_credit as t
group by t.sex'''
select(sql)

,sex,count(1),avg
0,female,310,2877.774194
1,male,690,3448.040580


**Задача 6. Расчет количества пропусков**

In [16]:
# чтобы и count отработал нормально - нужно использовать не 0, a null

sql = '''select t.checking_account,
count(case when t.checking_account is null then 1 else null end)
from german_credit as t'''

select(sql)



,checking_account,count(case when t.checking_account is null then 1 else null end)
0,None,394


**Задача 7. Посчитать количество кредитов помесячно.**

In [17]:
sql = '''select date(t.contract_dt, 'start of month') as month,
count(1) as credit_cnt
from german_credit as t
group by date(t.contract_dt, 'start of month')
'''

select(sql)

,month,credit_cnt
0,2007-05-01,81
1,2007-06-01,74
2,2007-07-01,71
3,2007-08-01,57
4,2007-09-01,58
5,2007-10-01,70
6,2007-11-01,87
7,2007-12-01,77
8,2008-01-01,93
9,2008-02-01,55


**Задача 8. Группировка по непрерывным переменным.**

In [18]:
# поэтому непрерывные переменные разбивают на диапазоны-интервалы-бакеты

sql = '''select t.credit_amount,
case when t.credit_amount < 1000 then '1. < 1000' 
when t.credit_amount between 1000 and 2000 then '2. 1000~2000'
when t.credit_amount between 2000 and 3000 then '3. 2000-3000'
when t.credit_amount between 3000 and 4000 then '4. 3000-4000'
when t.credit_amount between 4000 and 5000 then '5. 4000-5000'
when t.credit_amount between 5000 and 6000 then '6. 5000-6000'
when t.credit_amount between 6000 and 7000 then '7. 6000-7000'
else 'other' end as credit_amount_bin, count(*) as cnt

from german_credit as t
group by credit_amount_bin

'''

select(sql)

,credit_amount,credit_amount_bin,cnt
0,936,1. < 1000,116
1,1344,2. 1000~2000,316
2,2576,3. 2000-3000,188
3,3074,4. 3000-4000,134
4,4370,5. 4000-5000,58
5,5103,6. 5000-6000,39
6,6527,7. 6000-7000,44
7,14027,other,105


**Задача 9. Группировка переменной, заполненной в свободной форме.**

In [19]:
# свободная форма : в машине, машину, автомобиль, у машины, у автомобиля.... и так далее
# давай создадим df для эксперимента

df = pd.DataFrame({'purpose' : ['машина', 'машина','машина' , 'на машину', 
                                'на покупку машины', 'автомобиль','на возвращение 2007',
                                'на свадьбу', 'свадьба',
                               'свадьба', 'свадьба', 'для свадьбы',
                               'недвижимость',
                               'на покупку недвижимости']})

df

,purpose
0,машина
1,машина
2,машина
3,на машину
4,на покупку машины
5,автомобиль
6,на возвращение 2007
7,на свадьбу
8,свадьба
9,свадьба


In [20]:
df.to_sql('purpose', con, index=False, if_exists='replace')


14

In [21]:
sql = '''select t.purpose,
case when t.purpose like ('%машин%') then 1 else "other" end as 'машина'
from purpose as t'''

select(sql)

,purpose,машина
0,машина,1
1,машина,1
2,машина,1
3,на машину,1
4,на покупку машины,1
5,автомобиль,other
6,на возвращение 2007,other
7,на свадьбу,other
8,свадьба,other
9,свадьба,other


In [22]:
sql = ''' select case when t.purpose like ('%машин%') then 'машина'
when t.purpose like ('%авто%') then 'машина'
when t.purpose like ('%свадь%') then 'свадьба'
else 'другое' end as 'category', count(*)
from purpose as t
group by category'''

select(sql)

,category,count(*)
0,другое,3
1,машина,6
2,свадьба,5


In [23]:
# и теперь мы начинаем работать итерациями! мы берем и смотрим, что у нас осталось в категории other, и после
# этого в этой категории мы находим следующее самое частое значение, ЭТО ОДНАЯ ИТЕРАЦИЯ!!!!
# и для этого нам нужно взять уже размеченные данные, там где есть уже категорию 'другое', я их создал выше

# и мы сделаем это с помощью подзапроса

sql = '''select t.purpose,  count(1) 
from (select t.purpose, case when t.purpose like '%машин%' then 1 else 'other' end 
as purpose_cut from purpose as t) as t
where t.purpose_cut = 'other'
group by 1
order by count(1) desc'''

select(sql)

,purpose,count(1)
0,свадьба,3
1,недвижимость,1
2,на свадьбу,1
3,на покупку недвижимости,1
4,на возвращение 2007,1
5,для свадьбы,1
6,автомобиль,1


**Задача 10. Выполнить прошлую задачу через CTE. Объединить категории**

In [24]:
sql = '''
with category as (select t.purpose, 
case when (t.purpose like '%авто%' or  t.purpose like '%машин%') then 'машина' 
when t.purpose like '%свадь%' then 'свадьба'
when t.purpose like '%недвиж%' then 'недвижимость' 
else 'other' end as purpose_cat
from purpose as t)


select t.purpose_cat, count(1) from category as t
group by 1
'''

select(sql)

,purpose_cat,count(1)
0,other,1
1,машина,6
2,недвижимость,2
3,свадьба,5
